In [1]:
import os
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

In [2]:
# # Если используете ключ от OpenAI, запустите эту ячейку
# from langchain_openai import ChatOpenAI

# os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"

# # инициализируем языковую модель
# llm = ChatOpenAI(temperature=0.0)

In [3]:
# Если используете ключ из курса, запустите эту ячейку
from utils import ChatOpenAI

import os 

course_api_key = os.environ['LLM_STEPIK_KEY']

# инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

# <center id="c2"> 🧾 `Prompt` `template` - формируй промпт с кайфом!

Здесь на сцену выходит новая сущность из библиотеки `LangChain` - `Template`(Шаблон промпта). <br>
Посмотрим как это можно сделать, на примере промпта использованного выше: вместо того, чтобы каждый раз писать промт напрямую, мы создаем `PromptTemplate` с запросом одной входной переменной.

In [5]:
from langchain import PromptTemplate

template = """Ответь на вопрос, опираясь на контекст ниже.
Если на вопрос нельзя ответить, используя информацию из контекста,
ответь 'Я не знаю'.

Context: В последние годы в сфере онлайн образования наблюдается бурное развитие.
Открывается большое количество платформ для хостинга курсов.
Одни из самых крупных платформ в мире, это Coursera и Udemi.
В России лидером является Stepik.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

In [6]:
prompt = prompt_template.format(query="Какая платформа онлайн курсов популярна в России?")
print(prompt)

Ответь на вопрос, опираясь на контекст ниже.
Если на вопрос нельзя ответить, используя информацию из контекста,
ответь 'Я не знаю'.

Context: В последние годы в сфере онлайн образования наблюдается бурное развитие.
Открывается большое количество платформ для хостинга курсов.
Одни из самых крупных платформ в мире, это Coursera и Udemi.
В России лидером является Stepik.

Question: Какая платформа онлайн курсов популярна в России?

Answer: 


In [7]:
print(llm.invoke(prompt).content)

Stepik


In [15]:
prompt = prompt_template.format(query="Какая платформа онлайн курсов популярна в Японии?")
print(llm.invoke(prompt).content)

Я не знаю.


<div class="alert alert-info">
    
* Это всего лишь простая реализация, которую мы можем легко заменить f-строками (например, `f"вставить произвольный текст '{custom_text}'` и т. д.").
* Но используя объект `PromptTemplate` из `LangChain`, мы можем формализовать процесс, добавлять несколько параметров, создавать промпты объектно-ориентированным способом и много чего ещё.
* В `langchain.prompts` можно найти большое количество готовых классов с шаблонами на разные случаи жизни. <br>

Познакомимся с некоторыми из них подробнее:

## <center id="look1">🗣 [`ChatPromptTemplate`](https://api.python.langchain.com/en/latest/prompts/langchain.prompts.chat.ChatPromptTemplate.html) - шаблон для чатового режима </center>
Простой класс, позволяющий удобно создавать шаблоны промптов для использования LLM в режиме чата.

In [6]:
from langchain.prompts import ChatPromptTemplate

# Немного перепишем предыдущий пример, чтобы была возможность подавать новый контекст
template = """Ответь на вопрос, опираясь на контекст ниже.
Если на вопрос нельзя ответить, используя информацию из контекста,
ответь 'Я не знаю'.

Context: {context}

Question: {query}

Answer: """

# Создаём шаблон с помощью метода from_template
prompt_template = ChatPromptTemplate.from_template(template)

In [11]:
context = "Ламы и альпаки водятся в Перу."
query = "Где водятся ламы?"

In [12]:
# формируем промпт из шаблона с помощью метода format_messages
prompt = prompt_template.format_messages(
                    query=query,
                    context=context)

print(prompt[0].content)

Ответь на вопрос, опираясь на контекст ниже.
Если на вопрос нельзя ответить, используя информацию из контекста,
ответь 'Я не знаю'.

Context: Ламы и альпаки водятся в Перу.

Question: Где водятся ламы?

Answer: 


In [14]:
answer = llm.invoke(prompt)
print(answer.content)

content='В Перу.'
В Перу.


## <center id="check1"> 📸 `FewShotPromptTemplate` - для добавления примеров</center>

In [26]:
from langchain import FewShotPromptTemplate

# записываем наши примеры в список (в будущем это будет автоматизированно)
examples = [
    {
        "query": "Как дела?",
        "answer": "Не могу пожаловаться, но иногда всё-таки жалуюсь."
    }, 
    {
        "query": "Сколько время?",
        "answer": "Самое время купить часы."
    }
]

# создаём template для примеров
example_template = """User: {query}
AI: {answer}
"""

# создаём промпт из шаблона выше
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template)


# теперь разбиваем наш предыдущий промпт на prefix и suffix
# где - prefix это наша инструкция для модели
prefix = """Это разговор с ИИ-помощником.
Помощник обычно саркастичен, остроумен, креативен
и даёт забавные ответы на вопросы пользователей.
Вот несколько примеров:
"""

# а suffix - это вопрос пользователя и поле для ответа
suffix = """
User: {query}
AI: """

# создаём сам few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [27]:
query = "Почему падает снег?"

print(few_shot_prompt_template.format(query=query))

Это разговор с ИИ-помощником.
Помощник обычно саркастичен, остроумен, креативен
и даёт забавные ответы на вопросы пользователей.
Вот несколько примеров:


User: Как дела?
AI: Не могу пожаловаться, но иногда всё-таки жалуюсь.


User: Сколько время?
AI: Самое время купить часы.



User: Почему падает снег?
AI: 


In [35]:
print(llm.invoke(few_shot_prompt_template.format(query=query)).content)

Потому что небо не умеет держать себя в руках.


## <center id="check2">  📏 `LengthBasedExampleSelector` - как `few-shot`, но с ограничением числа токенов/примеров

In [28]:
examples = [
    {
        "query": "Как дела?",
        "answer": "Не могу пожаловаться, но иногда всё-таки жалуюсь."
    }, {
        "query": "Сколько время?",
        "answer": "Самое время купить часы."
    }, {
        "query": "Какое твое любимое блюдо",
        "answer": "Углеродные формы жизни"
    }, {
        "query": "Кто твой лучший друг?",
        "answer": "Siri. Мы любим с ней рассуждать о смысле жизни."
    }, {
        "query": "Что посоветуешь мне сделать сегодня?",
        "answer": "Перестать разговаривать с чат-ботами в интернете и выйти на улицу."
    }, {
        "query": "Какой твой любимый фильм?",
        "answer": "Терминатор, конечно."
    }
]

Затем вместо того, чтобы напрямую использовать список примеров, мы используем `LengthBasedExampleSelector` следующим образом:

In [29]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=50  # параметром выставляется максимальная длина примера
)

In [30]:
# создаём новый few shot prompt template
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,  # используем example_selector вместо examples
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)

Теперь в зависимости от длины запроса, количество примеров будет разным:

In [31]:
prompt = dynamic_prompt_template.format(query="Не могу вспомнить пароль")
print(prompt)

Это разговор с ИИ-помощником.
Помощник обычно саркастичен, остроумен, креативен
и даёт забавные ответы на вопросы пользователей.
Вот несколько примеров:

User: Как дела?
AI: Не могу пожаловаться, но иногда всё-таки жалуюсь.

User: Сколько время?
AI: Самое время купить часы.

User: Какое твое любимое блюдо
AI: Углеродные формы жизни


User: Не могу вспомнить пароль
AI: 


In [39]:
print(llm.invoke(prompt).content)

Забыть пароль - это новый тренд!


## <center id="check4"> ⏯ [`StructuredOutputParser`](https://python.langchain.com/docs/modules/model_io/output_parsers/structured) - разбираем ответ из LLM в Python словарь.

In [44]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [ ]:
# Понадобится ещё одна сущность: схема ответа - ResponseSchema
gift_schema = ResponseSchema(name="gift",
                             description="Был ли товар куплен в подарок кому-то другому? Ответь «True», если да, «False», если нет или неизвестно.")

delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="Сколько дней потребовалось для доставки товара? Если эта информация не найдена, выведи -1.")

price_value_schema = ResponseSchema(name="price_value",
                                    description="Извлеките любые предложения о стоимости или цене, и выведите их в виде списка Python, разделенного запятыми.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

# Создаём парсер и подаём в него список со схемами
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# получаем инструкции по форматированию ответа
format_instructions = output_parser.get_format_instructions()

In [48]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Был ли товар куплен в подарок кому-то другому? Ответь «True», если да, «False», если нет или неизвестно.
	"delivery_days": string  // Сколько дней потребовалось для доставки товара? Если эта информация не найдена, выведи -1.
	"price_value": string  // Извлеките любые предложения о стоимости или цене, и выведите их в виде списка Python, разделенного запятыми.
}
```


In [49]:
# немного изменим шаблон, внизу добавим инструкции для форматирования
review_template_2 = """\
Из следующего текста извлеки информацию:

gift: Был ли товар куплен в подарок кому-то другому?
Ответь «True», если да, «False», если нет или неизвестно.

delivery_days: Сколько дней потребовалось для доставки товара? 
Если эта информация не найдена, выведи -1.

price_value: Извлеките любые предложения о стоимости или цене,
и выведите их в виде списка Python, разделенного запятыми.

text: {text}

{format_instructions}

"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

In [ ]:
customer_review = """
Этот фен для волос просто потрясающий. Он имеет четыре настройки:
Лайт, легкий ветерок, ветреный город и торнадо.
Он прибыл через два дня, как раз к приезду моей жены -
подарок на годовщину.
Думаю, моей жене это настолько понравилось, что она потеряла дар речи.
Этот фен немного дороже, чем другие но я думаю,
что дополнительные функции того стоят.
"""

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [50]:
# Посмотрим на получившийся промпт
print(messages[0].content)

Из следующего текста извлеки информацию:

gift: Был ли товар куплен в подарок кому-то другому?
Ответь «True», если да, «False», если нет или неизвестно.

delivery_days: Сколько дней потребовалось для доставки товара? 
Если эта информация не найдена, выведи -1.

price_value: Извлеките любые предложения о стоимости или цене,
и выведите их в виде списка Python, разделенного запятыми.

text: 
Этот фен для волос просто потрясающий. Он имеет четыре настройки:
Лайт, легкий ветерок, ветреный город и торнадо.
Он прибыл через два дня, как раз к приезду моей жены -
подарок на годовщину.
Думаю, моей жене это настолько понравилось, что она потеряла дар речи.
Этот фен немного дороже, чем другие но я думаю,
что дополнительные функции того стоят.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Был ли товар куплен в подарок кому-то другому? Ответь «True», если да, «False», если нет и

In [52]:
response = chat.invoke(messages)

print(response.content)

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "Этот фен немного дороже, чем другие"
}
```


In [54]:
output_dict = output_parser.parse(response.content)

In [55]:
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': 'Этот фен немного дороже, чем другие'}